## Master Agreement/PSA Report 
* Goal: Make it easy for branch chiefs to know whether a Master Agreement and/or Program Supplement is needed or is going to expire in the next year.
* This will be a monthly report.


In [58]:
import A1_data_prep
import A2_tableau
import numpy as np
import pandas as pd
from babel.numbers import format_currency
from calitp import *

pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Project Sheet
* Column A - Award Year  
* Column B - Project #       
* Column C - Grant Recipient          
* Column D - Project Title 
* Column E - PPNO             
* Column I - Master Agreement Number
* Column J - Master Agreement Expiration Date
* Column K - Project Manager

#### Criteria
* If the master agreement field (column I Master Agreement Number, tab 1) is empty, the information should appear on the report
* If there is less than a year until the Master Agreement Expiration Date, the information should appear on the report

In [59]:
# Subset of cols
project_cols = [
    "project_award_year",
    "project_project_#",
    "project_grant_recipient",
    "project_project_title",
    "project_ppno",
    "project_master_agreement_number",
    "project_master_agreement_expiration_date",
    "project_project_manager",
]

In [60]:
# Tag projects with the appropriate comment
def ma_comments(row):

    if (row["project_master_agreement_number"] == "None") | (
        row["project_master_agreement_number"] == "Pending"
    ):
        return "None/Pending Master Agreement Number"
    else:
        return "<1 a year until Master Agreement Expiration Date"

In [61]:
def master_agreement(expiration_year: int):
    project = A1_data_prep.clean_project()

    # Subset df to only the columns requested
    project = project[project_cols]

    # Coerce to datetime
    project.project_master_agreement_expiration_date = (
        project.project_master_agreement_expiration_date.apply(
            pd.to_datetime, errors="coerce"
        )
    )

    # Conditions
    # If there is less than a year until the Master Agreement Expiration Date
    project_cond1 = (
        project.project_master_agreement_expiration_date.dt.year == expiration_year
    )
    # If the master agreement field (column I Master Agreement Number, tab 1) is empty
    project_cond2 = project.project_master_agreement_number.isin(["None", "Pending"])

    # Filter based on criteria above
    project = (project[(project_cond1 | project_cond2)]).reset_index(drop=True)

    project["Comments"] = project.apply(ma_comments, axis=1)

    return project

In [62]:
project_test = master_agreement(2023)

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [63]:
len(project_test)

16

### Allocation Sheet
* Column F - Project ID
* Column E - EA
* Column I - Phase 
* Column J - Allocation Amount
* Column T - Allocation Date 
* Column V - PSA #
* Column Q: Allocation Amendment
* Column AB - Date Branch Chief Receives PSA       
* Column AC - Date Regional Coordinator Receives PSA      
* Column AD - Date OC Receives PSA          
* Column AE - Date OPM Receives PSA      
* Column AF - Date Legal Receives PSA      
* Column AG - Date Returned to PM           
* Column AH - Date PSA Sent to Local Agency         
* Column AI - Date PSA Approved by Local Agency 
* Column AJ - Date Signed by DRMT
* Column AK – PSA Expiry Date


In [64]:
alloc = A1_data_prep.clean_allocation()

In [65]:
# Subset of cols
alloc_cols = [
    "allocation_grant_recipient",
    "allocation_ppno",
    "allocation_project_id",
    "allocation_ea",
    "allocation_phase",
    "allocation_allocation_amount",
    "allocation_allocation_date",
    "allocation_psa_#",
    "allocation_date_branch_chief_receives_psa",
    "allocation_date_regional_coordinator_receives_psa",
    "allocation_date_oc_receives_psa",
    "allocation_date_opm_receives_psa",
    "allocation_date_legal_receives_psa",
    "allocation_date_returned_to_pm",
    "allocation_date_psa_sent_to_local_agency",
    "allocation_date_psa_approved_by_local_agency",
    "allocation_ctc_allocation_amendment",
    "allocation_date_signed_by_drmt",
    "allocation_psa_expiry_date",
]

In [66]:
alloc2 = alloc[alloc_cols]

In [67]:
alloc2.isna().sum() / len(alloc2)

allocation_grant_recipient                          0.00
allocation_ppno                                     0.00
allocation_project_id                               0.00
allocation_ea                                       0.00
allocation_phase                                    0.00
allocation_allocation_amount                        0.00
allocation_allocation_date                          0.25
allocation_psa_#                                    0.00
allocation_date_branch_chief_receives_psa           0.97
allocation_date_regional_coordinator_receives_psa   1.00
allocation_date_oc_receives_psa                     0.93
allocation_date_opm_receives_psa                    0.67
allocation_date_legal_receives_psa                  0.66
allocation_date_returned_to_pm                      0.67
allocation_date_psa_sent_to_local_agency            0.00
allocation_date_psa_approved_by_local_agency        0.88
allocation_ctc_allocation_amendment                 0.00
allocation_date_signed_by_drmt 

In [68]:
# alloc2[['allocation_ctc_allocation_amendment']]

#### Criteria
* If there is a date in the “Allocation Date” field (column T) and no date in the “Date Signed by DRMT” (column AJ), then the information should be on the report
* If there is 6 months or less until the expiry date (column AK), then the information would be on the report


In [69]:
alloc_cond1 = (alloc2.allocation_allocation_date.notna()) & (
    alloc2.allocation_date_signed_by_drmt.isna()
)

In [70]:
# Replace with variables later not hard coded values
alloc_cond2 = (alloc2.allocation_psa_expiry_date.dt.date.astype(str) > "2022-12-31") & (
    alloc2.allocation_psa_expiry_date.dt.date.astype(str) < "2023-06-12"
)

In [71]:
alloc3 = (alloc2[alloc_cond1 | (alloc_cond2)]).reset_index(drop=True)

In [72]:
len(alloc3)

54

In [73]:
# alloc3[['allocation_allocation_date','allocation_date_signed_by_drmt','allocation_psa_expiry_date']]

##### If there is new information in column Q (allocation_ctc_allocation_amendment) and no change in column, then the information should be on the report
* https://stackoverflow.com/questions/54879260/how-to-highlight-differences-in-pandas-data-frame-after-concatenating-them

In [74]:
def load_previous_allocation():
    df = to_snakecase(
        pd.read_excel(
            f"{A1_data_prep.GCS_FILE_PATH}fake_allocation_sheet.xlsx",
            sheet_name="fake_aa",
        )
    )

    """
    Some rows are not completely filled: drop them based on whether or not some
    cols are populated.
    """
    df = df.dropna(subset=["award_year", "grant_recipient", "ppno"])

    # Fill in NA based on data type
    df.ctc_allocation_amendment = df.ctc_allocation_amendment.fillna("None")

    # Prefix to distinguish
    df = df.add_prefix("fake_allocation_")

    # Subset
    df = df[[
        "fake_allocation_ppno",
        "fake_allocation_grant_recipient",
        "fake_allocation_project_id",
        "fake_allocation_phase",
        "fake_allocation_ctc_allocation_amendment",
    ]]
    
    
    # Clean organization name/de duplicate
    df = A1_data_prep.organization_cleaning(df, "fake_allocation_grant_recipient")
    return df

In [75]:
alloc_fake_test = load_previous_allocation()

In [51]:
previous_alloc_merge_cols = ["fake_allocation_ppno",
    
        "fake_allocation_project_id",
        "fake_allocation_phase",]

In [52]:
current_alloc_merge_cols = ["allocation_ppno",
     
        "allocation_project_id",
        "allocation_phase",]

In [53]:
m1 = pd.merge(
    alloc2, alloc_fake_test, how="outer", left_on=current_alloc_merge_cols, 
    right_on=previous_alloc_merge_cols, indicator=True
)

In [54]:
len(m1)

631

In [55]:
m1._merge.value_counts()

right_only    233
left_only     227
both          171
Name: _merge, dtype: int64

In [56]:
preview_cols = ["fake_allocation_ctc_allocation_amendment","allocation_ctc_allocation_amendment", "_merge"]